In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [4]:
import pathlib

In [12]:
data_path = pathlib.Path("/home/evorsin/DS/mts_23/data/competition_data_final_pqt")
pqs_path_list = [item for item in data_path.iterdir() if not item.is_dir()]

In [13]:
pqs_path_list

[PosixPath('/home/evorsin/DS/mts_23/data/competition_data_final_pqt/part-00006-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/competition_data_final_pqt/part-00004-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/competition_data_final_pqt/part-00008-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/competition_data_final_pqt/part-00005-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/competition_data_final_pqt/part-00007-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/competition_data_final_pqt/part-00001-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/competition_data_final_pqt/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet'),
 PosixPath('/home/evorsin/D

# на первом паркете извлекаем сведения

In [7]:
data = pd.read_parquet(pqs_path_list[0])

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32134423 entries, 0 to 32134422
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   region_name            object 
 1   city_name              object 
 2   cpe_manufacturer_name  object 
 3   cpe_model_name         object 
 4   url_host               object 
 5   cpe_type_cd            object 
 6   cpe_model_os_type      object 
 7   price                  float64
 8   date                   object 
 9   part_of_day            object 
 10  request_cnt            int64  
 11  user_id                int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 2.9+ GB


In [9]:
data.date.head()

0    2021-07-08
1    2021-07-26
2    2021-07-09
3    2021-07-10
4    2021-08-01
Name: date, dtype: object

In [10]:
data.date.head().min()

datetime.date(2021, 7, 8)

In [11]:
data.date.head().max()

datetime.date(2021, 8, 1)

## по Клиентам извлекаем статы
* ИД
* мин дата
* макс дата
* разница в днях (после считаем)
* количество строк по клиенту

In [16]:
user_stats = data.groupby('user_id').agg({'date':['min','max'],'user_id':['count']}).reset_index()
user_stats.columns = ['user_id', 'min_date', 'max_date', 's_count']

In [17]:
user_stats

,user_id,min_date,max_date,s_count
0,8,2022-02-23,2022-06-19,976
1,25,2021-06-30,2021-07-18,252
2,38,2021-06-16,2021-08-23,5392
3,62,2021-07-07,2022-01-03,4789
4,72,2021-06-16,2021-07-24,819
...,...,...,...,...
41456,415268,2021-06-16,2021-07-28,19
41457,415297,2021-08-07,2021-08-07,2
41458,415300,2021-06-16,2021-06-16,4
41459,415307,2021-08-07,2021-08-07,1


In [23]:
prices = data.groupby(['cpe_manufacturer_name','cpe_model_name']).agg({'price':['min', 'max', 'mean']}).reset_index()
prices.columns = ['cpe_manufacturer_name','cpe_model_name', 'price_min', 'price_max', 'price_mean']

In [28]:
prices

,cpe_manufacturer_name,cpe_model_name,price_min,price_max,price_mean
0,Alcatel,1,3490.0,4990.0,3878.272478
1,Alcatel,1X,6823.0,7990.0,7110.963504
2,Alcatel,3x,NaN,NaN,NaN
3,Alcatel,U5 3G Dual,3990.0,3990.0,3990.000000
4,Apple,iPad Air,NaN,NaN,NaN
...,...,...,...,...,...
520,ZTE,Blade L210RU,5290.0,5290.0,5290.000000
521,ZTE,Blade L8,2889.0,4490.0,3850.110116
522,ZTE,Blade V10 Vita RU,NaN,NaN,NaN
523,ZTE,Blade V9,NaN,NaN,NaN


In [19]:
data.isna().sum()

region_name                   0
city_name                     0
cpe_manufacturer_name         0
cpe_model_name                0
url_host                      0
cpe_type_cd                   0
cpe_model_os_type             0
price                    654577
date                          0
part_of_day                   0
request_cnt                   0
user_id                       0
dtype: int64

## по фичам считаем уникальные внутри каждого файла

In [18]:
cols = ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
       'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'date',
       'part_of_day', 'request_cnt', 'user_id']

Index(['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
       'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'price', 'date',
       'part_of_day', 'request_cnt', 'user_id'],
      dtype='object')

In [32]:
agg = data['region_name'].value_counts()
agg = agg.reset_index()
agg.columns = ['region_name','region_name_cnt']

In [34]:
agg

,region_name,region_name_cnt
0,Москва,4531489
1,Краснодарский край,2532520
2,Московская область,2127212
3,Санкт-Петербург,2058942
4,Республика Башкортостан,1512755
...,...,...
75,Республика Калмыкия,12463
76,Чеченская Республика,10110
77,Ненецкий АО,7181
78,Республика Ингушетия,535


# Теперь для каждого паркета собираем сведения и объединяем

In [37]:
import gc
from tqdm import tqdm

In [40]:
from collections import defaultdict

In [36]:
cols = ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name',
       'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'date',
       'part_of_day', 'request_cnt', 'user_id']

In [45]:
del data

In [47]:
stats = defaultdict(list)
for pq_path in tqdm(pqs_path_list, total=len(pqs_path_list)):
    data = pd.read_parquet(pq_path)
    
    #по клиентам считаем статы
    user_stats = data.groupby('user_id').agg({'date':['min','max'],'user_id':['count']}).reset_index()
    user_stats.columns = ['user_id', 'min_date', 'max_date', 's_count']
    
    stats['user_stats'].append(user_stats)
    
    #по телефонам считаем цены
    prices = data.groupby(['cpe_manufacturer_name','cpe_model_name']).agg({'price':['min', 'max', 'mean']}).reset_index()
    prices.columns = ['cpe_manufacturer_name','cpe_model_name', 'price_min', 'price_max', 'price_mean']
    
    stats['prices_stats'].append(prices)
    
    #по фичам собираем стату
    for col in tqdm(cols, total=len(cols)):
        agg = data[col].value_counts()
        agg = agg.reset_index()
        agg.columns = [col,col+'_cnt']
        stats[col].append(agg)
    del data
    gc.collect()

100%|███████████████████████████████████████████| 10/10 [07:11<00:00, 43.15s/it]


## собираем в разрезе клиентов, цен статистики

In [48]:
stats.keys()

dict_keys(['user_stats', 'prices_stats', 'region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name', 'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'date', 'part_of_day', 'request_cnt', 'user_id'])

In [49]:
users_all = pd.concat(stats['user_stats'])

In [50]:
users_all.user_id

,user_id,min_date,max_date,s_count
0,8,2022-02-23,2022-06-19,976
1,25,2021-06-30,2021-07-18,252
2,38,2021-06-16,2021-08-23,5392
3,62,2021-07-07,2022-01-03,4789
4,72,2021-06-16,2021-07-24,819
...,...,...,...,...
41330,415257,2021-07-13,2021-07-18,9
41331,415285,2021-06-25,2021-07-09,3
41332,415291,2021-07-10,2021-07-10,1
41333,415304,2021-06-16,2021-06-16,2


In [54]:
prices_all = pd.concat(stats['prices_stats'])

In [63]:
prices_all_clean = prices_all[~prices_all.price_min.isna()].groupby(['cpe_manufacturer_name', 'cpe_model_name'])\
.agg({'price_min':['mean'],'price_max':['mean'],'price_mean':['mean'],}).reset_index()

In [65]:
prices_all.columns

Index(['cpe_manufacturer_name', 'cpe_model_name', 'price_min', 'price_max',
       'price_mean'],
      dtype='object')

In [66]:
prices_all_clean.columns = ['cpe_manufacturer_name', 'cpe_model_name', 'price_min', 'price_max',
       'price_mean']

In [68]:
prices_all_clean.sort_values('price_mean')

,cpe_manufacturer_name,cpe_model_name,price_min,price_max,price_mean
151,Meizu,M8,928.500000,965.250000,932.724909
47,BQ Devices Limited,BQ-5340 Choice,720.000000,2761.700000,1355.004756
261,Samsung,Galaxy J1 mini 3G Dual,858.000000,3408.100000,1574.446209
3,Alcatel,U5 3G Dual,1872.000000,1978.600000,1879.434279
154,Nokia,1 Dual,1676.000000,2200.571429,2095.079404
...,...,...,...,...,...
295,Samsung,Galaxy Note20 Ultra 5G Dual Global,111240.000000,119990.000000,116988.666008
14,Apple,iPhone 13 Pro Max,114531.900000,179190.300000,119329.407778
311,Samsung,Galaxy S22 Ultra,120444.666667,126475.666667,125252.718028
327,Samsung,Galaxy Z Fold3 5G,160877.333333,161488.833333,161462.538833


In [69]:
prices_all_clean.to_csv(d, index=False)

In [71]:
users_all.to_csv('/home/evorsin/DS/mts_23/mts_project_23/refs/users_all.csv', index=False)

## для кат фичей формируем маппинги для энгодинга

In [72]:
cols

['region_name',
 'city_name',
 'cpe_manufacturer_name',
 'cpe_model_name',
 'url_host',
 'cpe_type_cd',
 'cpe_model_os_type',
 'date',
 'part_of_day',
 'request_cnt',
 'user_id']

In [73]:
agg =pd.concat(stats['region_name'])

In [79]:
agg = agg.groupby('region_name')['region_name_cnt'].sum().reset_index().sort_values('region_name_cnt', ascending=False)

In [81]:
agg['region_name_id'] = range(1,len(agg)+1)

In [85]:
agg_dict = {k:v for k,v in zip(agg['region_name'].to_list(), agg['region_name_id'].to_list())}

In [87]:
stats_map = defaultdict(dict)

In [88]:
for col in cols:
    if col!='user_id':
        agg =pd.concat(stats[col])
        agg = agg.groupby(col)[col+'_cnt'].sum().reset_index().sort_values(col+'_cnt', ascending=False)
        agg[col+'_id'] = range(1,len(agg)+1)
        agg_dict = {k:v for k,v in zip(agg[col].to_list(), agg[col+'_id'].to_list())}
        
        stats_map[col]['stats']=agg
        stats_map[col]['col2id']=agg_dict

In [97]:
stats_map['url_host']['stats'].url_host_cnt.quantile(.99)

1881.0

In [3]:
from utils import save_pickle, load_pickle

In [ ]:
save_pickle()

In [4]:
import pickle

In [5]:
def save_pickle(path='filename.pickle', your_data=None):
    with open(path, 'wb') as handle:
        pickle.dump(your_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
def load_pickle(path='filename.pickle'):
    with open(path, 'rb') as handle:
        unserialized_data = pickle.load(handle)
    return unserialized_data

In [103]:
save_pickle('/home/evorsin/DS/mts_23/mts_project_23/refs/feats_stat.pickle', stats_map)

In [104]:
stats_map_t = load_pickle('/home/evorsin/DS/mts_23/mts_project_23/refs/feats_stat.pickle')

In [105]:
stats_map_t['url_host']['stats'].url_host_cnt.quantile(.99)

1881.0

# Выполним энкодинг и понизим размерность
* Для цены вставим среднее в пропуски
* Кат фичи меняем
* Генерим фичи по дате
* Понижаем размерность
* Удаляем старые фичи и сохраняем

In [6]:
stats_map_t = load_pickle('/home/evorsin/DS/mts_23/mts_project_23/refs/feats_stat.pickle')

In [7]:
stats_map_t.keys()

dict_keys(['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name', 'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'date', 'part_of_day', 'request_cnt'])

In [38]:
for col in stats_map_t.keys():
    print(col, len(stats_map_t[col]['col2id']))

region_name 81
city_name 985
cpe_manufacturer_name 37
cpe_model_name 599
url_host 199683
cpe_type_cd 4
cpe_model_os_type 3
date 396
part_of_day 4
request_cnt 15


In [14]:
data_path = pathlib.Path("/home/evorsin/DS/mts_23/data/competition_data_final_pqt")
pqs_path_list = [item for item in data_path.iterdir() if not item.is_dir()]

In [15]:
data = pd.read_parquet(pqs_path_list[0])

In [90]:
def preprocess(ch):
    prices = pd.read_csv('~/DS/mts_23/mts_project_23/refs/prices_all_clean.csv')
    median_prices = prices.price_mean.median()
    prices_mean = prices.groupby(['cpe_manufacturer_name']).price_mean.mean().reset_index()
    ch['price_nan'] = ch['price'].isna()*1
    ch['price_nan'] =ch['price_nan'].astype('int16') 
    phone_cls = ['cpe_manufacturer_name','cpe_model_name']
    price =pd.merge(ch[ch.price.isna()][phone_cls], 
        prices[phone_cls+['price_mean']].rename({'price_mean':'price'}, axis=1),
        on=phone_cls, how='left').price
    ch.loc[ch.price.isna(), 'price']=price
    price =pd.merge(ch[ch.price.isna()]['cpe_manufacturer_name'], 
        prices_mean.rename({'price_mean':'price'}, axis=1),
        on='cpe_manufacturer_name', how='left')
    ch.loc[ch.price.isna(), 'price']=price
    ch['price'].fillna(median_prices, inplace=True)
    mean_p = ch['price'].mean()
    std_p = ch['price'].std()
    #(value - mean) / stdev
    ch['price_sc'] = (ch['price']-mean_p)/std_p
    ch['price_sc'] =ch['price_sc'].astype('float32')
    ch['price'] =ch['price'].astype('float32')
    
    for col in ['region_name', 'city_name', 'cpe_manufacturer_name', 'cpe_model_name', 
                'url_host', 'cpe_type_cd', 'cpe_model_os_type', 'part_of_day']:
        ch[col+'_enc'] = ch[col].map(stats_map_t[col]['col2id'])
        if col=='url_host':
            ch[col+'_enc'] = ch[col+'_enc'].astype('int32')
        else:
            ch[col+'_enc'] = ch[col+'_enc'].astype('int16')
        ch.drop(col, axis=1, inplace=True)
        
    ch['request_cnt'] =ch['request_cnt'].astype('int16')
    ch['user_id']=ch['user_id'].astype('int32')
    
    ch['date'] = pd.to_datetime(ch['date'])
    ch['date_dow'] = ch['date'].dt.dayofweek.astype('int16') +1
    ch['date_day'] = ch['date'].dt.day.astype('int16')
    ch['date_month']=ch['date'].dt.month.astype('int16')
    
    ch.sort_values(['user_id','date'], inplace=True)
    return ch
    

In [100]:
for i, p_path in enumerate(tqdm(pqs_path_list, total=len(pqs_path_list))):
    data = pd.read_parquet(p_path)
    data = preprocess(data)
    data.to_parquet(f'/home/evorsin/DS/mts_23/data/encoded_data/part_{i}.parquet', index=False)

100%|███████████████████████████████████████████| 10/10 [08:49<00:00, 52.96s/it]


# Приведем в вертикальный вид и сохраним куски поменьше
важно - данные в разрезе клиента отсортированы по дате по возрастанию (ascending=True)
дело в том, что в нейронках буду брать последние N записей и чтобы отрезать именно последние нужна сортировка, а также, чтобы нейронки работали с последовательностью по клиенту хотя бы на уровне дней.

Ниже фичи константы для клиента, так что их вообще можно  брать в одном экземпляре для экономии места. 
'city_name_enc', 'cpe_manufacturer_name_enc',
       'cpe_model_name_enc', 'cpe_type_cd_enc',
       'cpe_model_os_type_enc', price, price_sc, price_nan

In [2]:
import gc

In [5]:
data_path = pathlib.Path("/home/evorsin/DS/mts_23/data/encoded_data/")
pqs_path_list = [item for item in data_path.iterdir() if not item.is_dir()]

In [7]:
pqs_path_list.sort()

In [8]:
pqs_path_list

[PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_0.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_1.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_2.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_3.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_4.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_5.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_6.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_7.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_8.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_9.parquet')]

In [5]:
data = pd.read_parquet(pqs_path_list[0])

In [30]:
def make_agg_sequence(data):
    agg = data.groupby('user_id').agg({'price':'first',
                                         'price_nan':'first',
                                         'price_sc':'first',
                                         'cpe_manufacturer_name_enc':'first',
                                         'cpe_model_name_enc':'first',
                                         'cpe_type_cd_enc':'first',
                                         'cpe_model_os_type_enc':'first',                                         
                                         'date':lambda x:x.tolist(),
                                         'request_cnt':lambda x:x.tolist(),
                                         'region_name_enc':lambda x:x.tolist(),
                                         'city_name_enc':lambda x:x.tolist(),
                                         'url_host_enc':lambda x:x.tolist(),
                                         'part_of_day_enc':lambda x:x.tolist(),
                                         'date_dow':lambda x:x.tolist(),
                                         'date_day':lambda x:x.tolist(),
                                         'date_month':lambda x:x.tolist(),
                                         'user_id':'count'}).rename({'user_id':'user_tr_count'}, axis=1).reset_index()
    del data
    gc.collect()
    return agg

In [35]:
for i, p_path in enumerate(tqdm(pqs_path_list, total=len(pqs_path_list))):
    data = pd.read_parquet(p_path)
    agg = make_agg_sequence(data)
    agg.to_parquet(f'/home/evorsin/DS/mts_23/data/encoded_data_vert/part_{i}.parquet', index=False)

100%|███████████████████████████████████████████| 10/10 [13:48<00:00, 82.80s/it]


# Разобьем по фолдам и добавим таргет

In [36]:
tgs = pd.read_csv('/home/evorsin/DS/mts_23/data/clean_users_tgs_cv.csv')

In [37]:
data_path = pathlib.Path("/home/evorsin/DS/mts_23/data/encoded_data_vert//")
pqs_path_list = [item for item in data_path.iterdir() if not item.is_dir()]
pqs_path_list.sort()

In [38]:
pqs_path_list

[PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_0.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_1.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_2.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_3.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_4.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_5.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_6.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_7.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_8.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data_vert/part_9.parquet')]

In [39]:
data = pd.read_parquet(pqs_path_list[0])

In [40]:
tgs

,user_id,min_date,max_date,s_count,split_data,age,is_male,diff_date_d,diff_date_m
0,8,2022-02-23,2022-06-19,976,fold_5,2.0,0.0,116.0,3.811167
1,25,2021-06-30,2021-07-18,252,fold_1,2.0,0.0,18.0,0.591388
2,38,2021-06-16,2021-08-23,5392,fold_4,1.0,1.0,68.0,2.234132
3,62,2021-07-07,2022-01-03,4789,fold_1,3.0,1.0,180.0,5.913879
4,72,2021-06-16,2021-07-24,819,subm,NaN,NaN,38.0,1.248486
...,...,...,...,...,...,...,...,...,...
408227,415257,2021-07-13,2021-07-18,9,fold_1,1.0,1.0,5.0,0.164274
408228,415285,2021-06-25,2021-07-09,3,subm,NaN,NaN,14.0,0.459968
408229,415291,2021-07-10,2021-07-10,1,subm,NaN,NaN,0.0,0.000000
408230,415304,2021-06-16,2021-06-16,2,fold_3,2.0,0.0,0.0,0.000000


In [42]:
data.shape

(41461, 18)

In [43]:
data = pd.merge(data, tgs, on ='user_id', how='inner')

In [44]:
data.shape

(40772, 26)

In [47]:
data.split_data.unique()

array(['fold_5', 'fold_1', 'fold_4', 'subm', 'fold_3', 'fold_2'],
      dtype=object)

In [45]:
data.split_data.value_counts()

subm      14485
fold_2     5340
fold_1     5318
fold_3     5248
fold_4     5196
fold_5     5185
Name: split_data, dtype: int64

In [61]:
for i, p_path in enumerate(tqdm(pqs_path_list, total=len(pqs_path_list))):
    data = pd.read_parquet(p_path)
    data = pd.merge(data, tgs, on ='user_id', how='inner')
    for fold in ['fold_5', 'fold_1', 'fold_4', 'subm', 'fold_3', 'fold_2']:
        data[data.split_data==fold].to_parquet(f'/home/evorsin/DS/mts_23/data/CV/{fold}/{fold}_part_{i}.parquet', index=False)
        

100%|███████████████████████████████████████████| 10/10 [01:39<00:00,  9.95s/it]


In [62]:
del data
gc.collect()

7640

In [63]:
for j in range(1,6):
    print(j)
    curr = [i for i in range(1,6) if i!=j]
    for l in curr:
        source = f'/home/evorsin/DS/mts_23/data/CV/fold_{l}/.'
        dest = f'/home/evorsin/DS/mts_23/data/CV/train/fold_{j}/'
        print(f'from fold_{l}/ TO fold_{j}')
        os.system(f'cp -a {source} {dest}')

1
from fold_2/ TO fold_1
from fold_3/ TO fold_1
from fold_4/ TO fold_1
from fold_5/ TO fold_1
2
from fold_1/ TO fold_2
from fold_3/ TO fold_2
from fold_4/ TO fold_2
from fold_5/ TO fold_2
3
from fold_1/ TO fold_3
from fold_2/ TO fold_3
from fold_4/ TO fold_3
from fold_5/ TO fold_3
4
from fold_1/ TO fold_4
from fold_2/ TO fold_4
from fold_3/ TO fold_4
from fold_5/ TO fold_4
5
from fold_1/ TO fold_5
from fold_2/ TO fold_5
from fold_3/ TO fold_5
from fold_4/ TO fold_5


In [51]:
import os

In [52]:
import sys

In [55]:
a = '/home/evorsin/DS/mts_23/data/CV/val'

In [54]:
!ls {a}

fold_1	fold_2	fold_3	fold_4	fold_5


In [57]:
os.system(f'ls {a}')

fold_1
fold_2
fold_3
fold_4
fold_5


0

In [64]:
data_path = pathlib.Path("/home/evorsin/DS/mts_23/data/CV/train/fold_5")
pqs_path_list = [item for item in data_path.iterdir() if not item.is_dir()]
pqs_path_list.sort()

In [67]:
f3 = pd.read_parquet('/home/evorsin/DS/mts_23/data/CV/fold_3')

In [68]:
f3

,user_id,price,price_nan,price_sc,cpe_manufacturer_name_enc,cpe_model_name_enc,cpe_type_cd_enc,cpe_model_os_type_enc,date,request_cnt,...,date_month,user_tr_count,min_date,max_date,s_count,split_data,age,is_male,diff_date_d,diff_date_m
0,87,34990.0,0,0.094888,3,134,1,1,"[2021-06-16T00:00:00.000000, 2021-06-16T00:00:...","[1, 1, 2, 1, 3, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, ...",...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...",1105,2021-06-16,2021-07-01,1105,fold_3,1.0,0.0,15.0,0.492823
1,182,7987.0,0,-0.961281,4,80,1,1,"[2021-06-16T00:00:00.000000, 2021-06-16T00:00:...","[2, 2, 2, 3, 1, 2, 4, 2, 2, 2, 3, 5, 4, 1, 1, ...",...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...",1256,2021-06-16,2021-07-31,1256,fold_3,3.0,1.0,45.0,1.478470
2,215,56734.0,0,0.945363,1,2,1,2,"[2022-02-23T00:00:00.000000, 2022-02-23T00:00:...","[1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, ...",...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",600,2022-02-23,2022-06-19,600,fold_3,6.0,1.0,116.0,3.811167
3,235,14938.0,0,-0.689406,4,51,1,1,"[2021-06-16T00:00:00.000000, 2021-06-16T00:00:...","[4, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, ...",...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...",1005,2021-06-16,2021-08-26,1005,fold_3,3.0,1.0,71.0,2.332697
4,304,72387.0,0,1.557599,1,9,1,2,"[2021-06-16T00:00:00.000000, 2021-06-16T00:00:...","[1, 1, 3, 1, 2, 2, 1, 1, 1, 1, 3, 1, 3, 1, 1, ...",...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...",2125,2021-06-16,2021-07-28,2125,fold_3,1.0,1.0,42.0,1.379905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52690,414930,35748.0,0,0.119159,1,3,1,2,"[2021-06-16T00:00:00.000000, 2021-06-16T00:00:...","[2, 1, 2, 3, 4, 2, 3, 3, 2, 3, 3, 1, 1, 6, 1, ...",...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...",142,2021-06-16,2021-06-28,142,fold_3,5.0,0.0,12.0,0.394259
52691,414984,13063.0,0,-0.760609,4,99,1,1,"[2021-06-19T00:00:00.000000, 2021-06-20T00:00:...","[1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, ...",...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...",63,2021-06-19,2021-07-14,63,fold_3,4.0,0.0,25.0,0.821372
52692,415084,7786.0,0,-0.965262,3,39,1,1,"[2021-06-25T00:00:00.000000, 2021-06-25T00:00:...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",...,"[6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]",14,2021-06-25,2021-07-29,14,fold_3,4.0,1.0,34.0,1.117066
52693,415188,13990.0,0,-0.724658,3,121,1,1,"[2021-06-16T00:00:00.000000, 2021-06-16T00:00:...","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",...,"[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]",12,2021-06-16,2021-06-16,12,fold_3,5.0,1.0,0.0,0.000000


In [9]:
data = pd.read_parquet(pqs_path_list[0])

In [10]:
pqs_path_list[1:]

[PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_1.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_2.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_3.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_4.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_5.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_6.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_7.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_8.parquet'),
 PosixPath('/home/evorsin/DS/mts_23/data/encoded_data/part_9.parquet')]

In [11]:
import gc

In [12]:
for p in pqs_path_list[1:]:
    t_d = pd.read_parquet(p)
    data=pd.concat([data,t_d], ignore_index=True)
    del t_d
    gc.collect()


In [13]:
data.shape

(322899435, 17)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322899435 entries, 0 to 322899434
Data columns (total 17 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   price                      float32       
 1   date                       datetime64[ns]
 2   request_cnt                int16         
 3   user_id                    int32         
 4   price_nan                  int16         
 5   price_sc                   float32       
 6   region_name_enc            int16         
 7   city_name_enc              int16         
 8   cpe_manufacturer_name_enc  int16         
 9   cpe_model_name_enc         int16         
 10  url_host_enc               int32         
 11  cpe_type_cd_enc            int16         
 12  cpe_model_os_type_enc      int16         
 13  part_of_day_enc            int16         
 14  date_dow                   int16         
 15  date_day                   int16         
 16  date_month                 int16

In [1]:
250000/512

488.28125